# Stability Verification for an Inverted Pendulum

In [ ]:
from __future__ import division, print_function

import numpy as np
import tensorflow as tf
import gpflow
import safe_learning
import matplotlib.pyplot as plt
import time
import pandas
import os

from scipy.linalg import block_diag
from utilities import InvertedPendulum, debug
from safe_learning.utilities import get_storage, set_storage
from matplotlib.font_manager import FontProperties
from matplotlib.colors import ListedColormap

%matplotlib inline
    
NP_DTYPE = safe_learning.config.np_dtype
TF_DTYPE = safe_learning.config.dtype
_STORAGE = {}
EPS = 1e-8

HEAT_MAP = plt.get_cmap('inferno', lut=None)
HEAT_MAP.set_over((1., 1., 1., 0.))
HEAT_MAP.set_under('black')

LEVEL_MAP = plt.get_cmap('viridis', lut=15)
LEVEL_MAP.set_over('gold')
LEVEL_MAP.set_under('white')

BINARY_MAP = ListedColormap([(1., 1., 1., 0.), (0., 1., 0., 0.65)])

pandas.options.display.float_format = '{:,.4f}'.format
pandas.set_option('expand_frame_repr', False)
np.set_printoptions(precision=4)


# TODO testing ****************************************#

class Options(object):
    def __init__(self, **kwargs):
        super(Options, self).__init__()
        self.__dict__.update(kwargs)

OPTIONS = Options(np_dtype              = safe_learning.config.np_dtype,
                  tf_dtype              = safe_learning.config.dtype,
                  use_linear_dynamics   = False,
                  saturate              = True,
                  eps                   = 1e-8,
                  dpi                   = 100,
                  fontproperties        = FontProperties(size=16),
                  save_figs             = False
                 )

## TensorFlow Session

In [ ]:
MAX_CPU_COUNT = os.cpu_count()
NUM_CORES = 8
NUM_SOCKETS = 2

os.environ["KMP_BLOCKTIME"]    = str(0)
os.environ["KMP_SETTINGS"]     = str(1)
os.environ["KMP_AFFINITY"]     = 'granularity=fine,noverbose,compact,1,0'
os.environ["OMP_NUM_THREADS"]  = str(NUM_CORES)

config = tf.ConfigProto(intra_op_parallelism_threads  = NUM_CORES,
                        inter_op_parallelism_threads  = NUM_SOCKETS,
                        allow_soft_placement          = False,
#                         log_device_placement          = True,
                        device_count                  = {'CPU': MAX_CPU_COUNT},
                       )

# TODO manually for CPU-only?
config.graph_options.optimizer_options.global_jit_level = tf.OptimizerOptions.ON_1

try:
    session.close()
except NameError:
    pass
session = tf.InteractiveSession(config=config)

# print('Found MAX_CPU_COUNT =', MAX_CPU_COUNT)
# for dev in session.list_devices():
#     print(dev)

## Flags

In [ ]:
# Saturate the action so that it lies in [-1, 1]
SATURATE = True

# Use the true physical parameters in the GP model
USE_TRUE_PARAMETERS = False

# Use the linearized discrete-time model as the true underlying dynamics
USE_LINEAR_DYNAMICS = False

# Use a threshold of zero when checking for stability
USE_ZERO_THRESHOLD = False

#
USE_LINEAR_KERNELS = False

#
USE_LIPSCHITZ_SCALING = True

# Scaling factor for GP confidence intervals
BETA = 2.

#
TRAIN_HYPERPARAMETERS = False

#
GP_SCALING = 1.

#
NOISE_VAR = 0.001 ** 2

#
ADAPTIVE = True


## Dynamics

In [ ]:
# Constants
dt = 0.01   # sampling time
g = 9.81    # gravity

# True system parameters
m = 0.15    # pendulum mass
L = 0.5     # pole length
b = 0.1     # rotational friction

# State and action normalizers
theta_max = np.deg2rad(30)
omega_max = np.sqrt(g / L)
u_max = g * m * L * np.sin(theta_max)

state_norm = (theta_max, omega_max)
action_norm = (u_max, )

# Constraints for initial 'safe' states
theta_safe = np.deg2rad(8)
omega_safe = 0.5*np.sqrt(g / L)

# Dimensions and domains
state_dim = 2
action_dim = 1
state_limits = np.array([[-1., 1.]] * state_dim)
action_limits = np.array([[-1., 1.]] * action_dim)

# True system
true_pendulum = InvertedPendulum(m, L, b, dt, [state_norm, action_norm])
A_true, B_true = true_pendulum.linearize()

if USE_LINEAR_DYNAMICS:
    true_dynamics = safe_learning.functions.LinearSystem((A_true, B_true), name='true_dynamics')
else:
    true_dynamics = true_pendulum.__call__

# "Wrong" system
m = 0.1     # pendulum mass
L = 0.4     # pole length
b = 0.0     # rotational friction
pendulum = InvertedPendulum(m, L, b, dt, [state_norm, action_norm])
A, B = pendulum.linearize()

if USE_TRUE_PARAMETERS:
    A = A_true
    B = B_true
mean_dynamics = safe_learning.LinearSystem((A, B), name='mean_dynamics')

## GP Model

In [ ]:
m_true = np.hstack((A_true, B_true))
m = np.hstack((A, B))
variances = (m_true - m) ** 2

# Make sure at least some non-zero prior variance is maintained
np.clip(variances, 1e-3, None, out=variances)

# Measurement noise
noise_var = NOISE_VAR

# Input to GP is of the form (x,u)
full_dim = state_dim + action_dim

# Kernels
if USE_LINEAR_KERNELS:
    kernel_theta = gpflow.kernels.Linear(full_dim, variance=variances[0, :], ARD=True)

    kernel_omega = gpflow.kernels.Linear(full_dim, variance=variances[1, :], ARD=True)

else:
    kernel_theta = (gpflow.kernels.Linear(full_dim, variance=variances[0, :], ARD=True)
                    + gpflow.kernels.Matern32(1, lengthscales=1, active_dims=[0])
                    * gpflow.kernels.Linear(1, variance=variances[0, 1]))

    kernel_omega = (gpflow.kernels.Linear(full_dim, variance=variances[1, :], ARD=True)
                    + gpflow.kernels.Matern32(1, lengthscales=1, active_dims=[0])
                    * gpflow.kernels.Linear(1, variance=variances[1, 1]))

# Mean dynamics
mean_function_theta = safe_learning.LinearSystem((A[[0], :], B[[0], :]), name='mean_dynamics_theta')
mean_function_omega = safe_learning.LinearSystem((A[[1], :], B[[1], :]), name='mean_dynamics_omega')

In [ ]:
# Define a GP model over the dynamics
X_init = np.zeros((1, full_dim), dtype=NP_DTYPE)
Y_init = np.zeros((1, 1), dtype=NP_DTYPE)

gp_theta = gpflow.gpr.GPR(X_init, Y_init, kernel_theta, mean_function_theta)
gp_omega = gpflow.gpr.GPR(X_init, Y_init, kernel_omega, mean_function_omega)


# TODO Tensorflow spits out a lot of allocator errors when creating 0-length dataholders in gpflow. Occurs when:
#     - initializing with empty data matrices X and Y
#     - using GPRCached (initializes empty dataholders for Cholesky decomposition)

# X_init = np.empty((0, full_dim), dtype=NP_DTYPE)
# Y_init = np.empty((0, 1), dtype=NP_DTYPE)
# gp_theta = safe_learning.GPRCached(X_init, Y_init, kernel_theta, mean_function_theta)
# gp_omega = safe_learning.GPRCached(X_init, Y_init, kernel_omega, mean_function_omega)

In [ ]:
gp_theta.likelihood.variance = noise_var
gp_omega.likelihood.variance = noise_var

gp_theta_fun = safe_learning.GaussianProcess(gp_theta, BETA)
gp_omega_fun = safe_learning.GaussianProcess(gp_omega, BETA)

# Stack GP functions => block-diagonal kernel matrix
dynamics = safe_learning.FunctionStack((gp_theta_fun, gp_omega_fun))

## State Discretization

In [ ]:
# Number of states along each dimension
if ADAPTIVE:
    num_states = 501
else:
    num_states = 3001

# State grid
limits = np.array([[-1., 1.], [-1., 1.]])
grid = safe_learning.GridWorld(limits, num_states)

# Discretization constant
if USE_ZERO_THRESHOLD:
    disc_constant = 0.0
else:
    disc_constant = np.sum(grid.unit_maxes) / 2

print('Grid size: {}'.format(grid.nindex))
print('Discretization constant: {}'.format(disc_constant))

## Policy

Fix the policy to the LQR solution for the true system, possibly with saturation constraints.

In [ ]:
# State cost matrix
Q = np.diag([1., 2.])

# Action cost matrix
R = 1.2*np.identity(action_dim)

# Normalize cost matrices
cost_norm = np.amax([Q.max(), R.max()])
Q = Q / cost_norm
R = R / cost_norm

# Quadratic cost function
cost_function = safe_learning.QuadraticFunction(block_diag(Q, R), name='cost_function')

# Solve the LQR problem for the true system
K, P = safe_learning.utilities.dlqr(A_true, B_true, Q, R)
policy = safe_learning.LinearSystem(-K, name='policy')

if SATURATE:
    policy = safe_learning.Saturation(policy, -1, 1)


In [ ]:
def plot_policy(policy, grid, norms, tol=1e-10):
    fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=OPTIONS.dpi)
    ticks = np.linspace(-1., 1., 9)
    cutoff = 1. - tol
    plot_limits = np.asarray(norms).reshape((-1, 1)) * grid.limits
    
    z = policy(grid.all_points).eval().reshape(grid.num_points)
    im = ax.imshow(z.T, origin='lower', extent=plot_limits.ravel(), aspect=plot_limits[0, 1] / plot_limits[1, 1], cmap=HEAT_MAP, vmin=-cutoff, vmax=cutoff)
    cbar = fig.colorbar(im, ax=ax, label=r'$u = \pi(x)$', ticks=ticks)
    ax.set_xlabel(r'$\phi$ [deg]')
    ax.set_ylabel(r'$\dot{\phi}$ [deg/s]')
    plt.show()

# Visualize policy
norms = np.rad2deg(state_norm)
plot_policy(policy, grid, norms)


## Lyapunov Function

In [ ]:
# Define the Lyapunov function corresponding to the known policy
lyapunov_function = safe_learning.QuadraticFunction(P)
grad_lyapunov_function = safe_learning.LinearSystem((2*P,))

# Lipschitz constants
L_pol = np.linalg.norm(-K, ord=1)
L_dyn = np.linalg.norm(A_true, ord=1) + np.linalg.norm(B_true, ord=1) * L_pol

if USE_LIPSCHITZ_SCALING:
    L_v = lambda x: tf.abs(grad_lyapunov_function(x))
else:
    L_v = lambda x: tf.norm(grad_lyapunov_function(x), ord=1, axis=1, keep_dims=True)
    
# Set initial safe set as a small level set of the Lyapunov function
values = lyapunov_function(grid.all_points).eval()
cutoff = 3e-3 * np.max(values)
initial_safe_set = np.squeeze(values, axis=1) <= cutoff

# Set initial safe set as a hypercube in the state space
# safe_norm = np.array([[theta_safe / theta_max, omega_safe / omega_max]])
# norm_states = state_discretization.all_points / safe_norm
# initial_safe_set = np.all(np.logical_and(norm_states >= -1, norm_states <= 1), axis=1, keepdims=False)

# Initialize class
lyapunov = safe_learning.Lyapunov(grid, lyapunov_function, dynamics, L_dyn, L_v, disc_constant, policy, initial_safe_set, adaptive=ADAPTIVE)
lyapunov.update_values()


## TensorFlow Graph

In [ ]:
storage = get_storage(_STORAGE)
if storage is None:
    # Current
    states = tf.placeholder(OPTIONS.tf_dtype, shape=[None, lyapunov.discretization.ndim], name='states')
    actions = policy(states)
    values = lyapunov.lyapunov_function(states)
    
    # Predicted future
    future_states_mean, future_states_error = lyapunov.dynamics(states, actions)
    future_values_mean = lyapunov.lyapunov_function(future_states_mean)
    lv = lyapunov.lipschitz_lyapunov(future_states_mean)
    future_values_error = tf.reduce_sum(lv * future_states_error, axis=1, keepdims=True)
    dv_mean = future_values_mean - values
    dv_bound = dv_mean + future_values_error
    
    # True future
    future_states = true_dynamics(states, actions)
    future_values = lyapunov.lyapunov_function(future_states)
    dv = future_values - values
    
    # Discretization effects
    tau = tf.placeholder(OPTIONS.tf_dtype, shape=[None, 1], name='discretization_constant')
    threshold = lyapunov.threshold(states, tau)
    negative = tf.less(dv_bound, threshold)
    
    # Place into storage
    storage = [('states', states), ('actions', actions), ('values', values), 
               ('future_states', future_states), ('future_values', future_values), ('dv', dv),
               ('tau', tau), ('threshold', threshold), ('negative', negative)]
    set_storage(_STORAGE, storage)
else:
    # Get from storage
    states, actions, values, future_states, future_values, dv, tau, threshold, negative  = storage.values()


## Visualize Discretization Effects

In [ ]:
fontsize = 18
plt.rc('font', size=fontsize)

feed_dict = {states: grid.all_points, tau: [[lyapunov.tau]]}
plot_limits = np.asarray(norms).reshape((-1, 1)) * grid.limits

fig, axes = plt.subplots(2, 2, figsize=(20, 15), dpi=OPTIONS.dpi)
fig.subplots_adjust(wspace=0.15, hspace=0.3)
vmin = -2

ax = axes[0, 0]
z = dv.eval(feed_dict).reshape(grid.num_points)
im = ax.imshow(z.T, origin='lower', extent=plot_limits.ravel(), aspect=plot_limits[0, 1] / plot_limits[1, 1], cmap=HEAT_MAP, vmin=vmin, vmax=0)
cbar = fig.colorbar(im, ax=ax, label=r'$v(f_\pi(x)) - v(x)$', ticks=[-2, -1.5, -1, -0.5, 0])
ax.set_title(r'No discretization ($\tau = 0$)', fontsize=fontsize)
ax.set_xlabel(r'$\phi$ [deg]')
ax.set_ylabel(r'$\dot{\phi}$ [deg/s]')
yticks = cbar.ax.get_yticks()
tick_labels = ['{:.1f}'.format((1 - y) * vmin) for y in yticks]
tick_labels[-1] = r'$\geq 0$'
cbar.ax.set_yticklabels(tick_labels)


for ax, M in zip(axes.ravel()[1:], [3001, 1001, 501]):
    disc = safe_learning.GridWorld(grid.limits, M)
    feed_dict[states] = disc.all_points
    feed_dict[tau] = [[np.sum(disc.unit_maxes) / 2]]
    
    z = (dv - threshold).eval(feed_dict).reshape(disc.num_points)
    im = ax.imshow(z.T, origin='lower', extent=plot_limits.ravel(), aspect=plot_limits[0, 1] / plot_limits[1, 1], cmap=HEAT_MAP, vmin=vmin, vmax=0)
    cbar = fig.colorbar(im, ax=ax, label=r'$v(f_\pi(x)) - v(x) + L_{\Delta v}\tau$', ticks=[-2, -1.5, -1, -0.5, 0])
    ax.set_title(r'$M = {}$'.format(M - 1) + ',  ' + r'$|\mathcal{X}_\tau|$ = ' + r'{:.1e}'.format(disc.nindex) 
                 + ',  ' + r'$\tau$ = ' + r'{:.0e}'.format(np.sum(disc.unit_maxes) / 2), fontsize=fontsize)
    ax.set_xlabel(r'$\phi$ [deg]')
    ax.set_ylabel(r'$\dot{\phi}$ [deg/s]')
    
    yticks = cbar.ax.get_yticks()
    tick_labels = ['{:.1f}'.format((1 - y) * vmin) for y in yticks]
    tick_labels[-1] = r'$\geq 0$'
    cbar.ax.set_yticklabels(tick_labels)
    
# cbar.ax.get_ymajorticklabels()[0] = '5'
# fig.tight_layout()
plt.show()

if OPTIONS.save_figs:
    fig.savefig('figures/pendulum_basic_grid_effect.pdf', bbox_inches='tight')

In [ ]:
fontsize = 18
plt.rc('font', size=fontsize)

limits = np.array([[-1., 1.], [-1., 1.]])
grid = safe_learning.GridWorld(limits, 501)

Nmax = 16
cmap = plt.get_cmap('viridis', lut=Nmax)
cmap.set_over('gold')
cmap.set_under((1., 1., 1., 0.))

# Adaptive discretization
feed_dict = {states: grid.all_points, tau: [[np.sum(grid.unit_maxes) / 2]]}
N = (threshold / dv).eval(feed_dict)
N[np.isnan(N)] = -1
N[N < 0] = -1
N = np.ceil(N)

fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=OPTIONS.dpi)

z = N.reshape(grid.num_points)
im = ax.imshow(z.T, origin='lower', extent=plot_limits.ravel(), aspect=plot_limits[0, 1] / plot_limits[1, 1], cmap=cmap, vmin=0, vmax=Nmax)
cbar = fig.colorbar(im, ax=ax, label=r'$N({\bf x})$', ticks=np.arange(0, Nmax + 1, 2))
ax.set_title(r'$M = {}$'.format(grid.num_points[0] - 1) 
             + ',  ' + r'$|\mathcal{X}_\tau|$ = ' + r'{:.1e}'.format(grid.nindex) 
             + ',  ' + r'$\tau$ = ' + r'{:.0e}'.format(np.sum(grid.unit_maxes) / 2), 
             fontsize=fontsize)
ax.set_xlabel(r'$\phi$ [deg]')
ax.set_ylabel(r'$\dot{\phi}$ [deg/s]')

yticks = cbar.ax.get_yticks()
tick_labels = ['{:.0f}'.format(y * Nmax) for y in yticks]
tick_labels[-1] = r'$\geq {}$'.format(Nmax)
cbar.ax.set_yticklabels(tick_labels)

plt.show()

if OPTIONS.save_figs:
    fig.savefig('figures/pendulum_adaptive_Nreq.pdf', bbox_inches='tight')

## Initial Safe Set Visualization

In [ ]:
# Compare safe set before and after checking the decrease condition for the first time
c_max = lyapunov.feed_dict[lyapunov.c_max]
init_safe_set_size = np.sum(lyapunov.safe_set)

print('Before update ...')
print('c_max: {}'.format(c_max))
print('Safe set size: {}\n'.format(init_safe_set_size))

old_safe_set = np.copy(lyapunov.safe_set)
lyapunov.update_safe_set()

c_max = lyapunov.feed_dict[lyapunov.c_max]
init_safe_set_size = np.sum(lyapunov.safe_set)

print('After update ...')
print('c_max: {}'.format(c_max))
print('Safe set size: {}'.format(init_safe_set_size))

# debug(lyapunov, true_dynamics, state_norm, plot='pendulum')

## Online Learning and Exploration

In [ ]:
# action_variation = np.array([-0.01, -0.001, 0.0, 0.001, 0.01], dtype=NP_DTYPE).reshape((-1, 1))
action_variation = np.array([[0.]], dtype=NP_DTYPE)

with tf.name_scope('add_new_measurement'):
    full_dim = state_dim + action_dim 
    tf_max_state_action = tf.placeholder(TF_DTYPE, shape=[1, full_dim])
    tf_measurement = true_dynamics(tf_max_state_action)
    
def update_gp():
    """Update the GP model based on an actively selected data point."""
    
    # Get a new sample location
    max_state_action, _ = safe_learning.get_safe_sample(lyapunov, action_variation, action_limits, positive=True, num_samples=1000)
    
    # Obtain a measurement of the true dynamics
    lyapunov.feed_dict[tf_max_state_action] = max_state_action
    measurement = tf_measurement.eval(feed_dict=lyapunov.feed_dict)
    
    # Add the measurement to our GP dynamics
    lyapunov.dynamics.add_data_point(max_state_action, measurement)

In [ ]:
data_per_update = 10
safe_set_updates = 3

can_shrink          = True
n_max               = 16
safety_factor       = 1.
parallel_iterations = NUM_CORES

try:
    e = len(level) - 1
    level = np.concatenate((level, np.zeros(safe_set_updates)))
    safe_size = np.concatenate((safe_size, np.zeros(safe_set_updates)))
    temp = data_per_update * np.arange(1, safe_set_updates + 1) + data_size[-1]
    data_size = np.concatenate((data_size, temp))
except NameError:
    e = 0
    level = np.zeros(safe_set_updates + 1)
    level[0] = lyapunov.feed_dict[lyapunov.c_max]
    safe_size = np.zeros(safe_set_updates + 1)
    safe_size[0] = np.sum(lyapunov.safe_set)
    data_size = data_per_update * np.arange(safe_set_updates + 1)
    
for i in range(safe_set_updates):
    print('Iteration {} with c_max: {}'.format(e + i + 1, lyapunov.feed_dict[lyapunov.c_max]))

    start = time.time()
    for _ in range(data_per_update): 
        update_gp()
    end = time.time()
    duration_gp = end - start
    
    start = time.time()
    lyapunov.update_safe_set(can_shrink, n_max, safety_factor, parallel_iterations)
    end = time.time()
    duration_lyap = end - start
    
    level[e + i + 1] = lyapunov.feed_dict[lyapunov.c_max]
    safe_size[e + i + 1] = np.sum(lyapunov.safe_set)
    
    num_data = lyapunov.dynamics.functions[0].X.shape[0]
    num_disc = np.prod(lyapunov.discretization.num_points)
    num_safe = np.sum(lyapunov.safe_set)
    
    print("Data points collected: {}".format(num_data))
    print('Discretization size: {}'.format(num_disc))
    print('Safe set size: {} ({:.2f}%)'.format(num_safe, 100 * num_safe / num_disc))
    print('Growth: {}'.format(num_safe - init_safe_set_size))
    print('Duration (gp, avg): {}'.format(duration_gp / data_per_update))
    print('Duration (lyap): {}'.format(duration_lyap))
    print("NEW C_MAX: {}".format(lyapunov.feed_dict[lyapunov.c_max]))
    print('')

In [ ]:
fontsize = 18
plt.rc('font', size=fontsize)

N = np.copy(lyapunov._n)
num_states = len(N)
num_refined_states = np.sum(N[N > 1] ** state_dim)
print('Grid size:', num_states)
print('Safe set size:', num_safe)
print('Refined grids size:', num_refined_states)
print('Effective total grid size:', num_refined_states + num_states)
print('Effective safe grid size:', num_refined_states + num_safe)

# debug(lyapunov, true_dynamics, state_norm, Nmax=128, plot='pendulum')

#
disc = lyapunov.discretization
feed_dict = {states: disc.all_points, tau: [[lyapunov.tau]]}
plot_limits = np.asarray(norms).reshape((-1, 1)) * disc.limits

#
fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=OPTIONS.dpi)

#
decrease_region = (dv.eval(feed_dict) < 0).reshape(disc.num_points)
cmap = ListedColormap([(1., 1., 1., 0.), 'lightgrey'])
im = ax.imshow(decrease_region.T, origin='lower', extent=plot_limits.ravel(), aspect=plot_limits[0, 1] / plot_limits[1, 1], cmap=cmap, vmin=0, vmax=None)

#
N = np.copy(lyapunov._n)
vals = values.eval(feed_dict).ravel()

# TODO
N[np.logical_and(vals <= lyapunov.feed_dict[lyapunov.c_max], N <= 0)] = 1
N[N == 0] = -1

z = N.reshape(disc.num_points)
cmap = plt.get_cmap('viridis', lut=n_max)
cmap.set_over('gold')
cmap.set_under((1., 1., 1., 0.))
im = ax.imshow(z.T, origin='lower', extent=plot_limits.ravel(), aspect=plot_limits[0, 1] / plot_limits[1, 1], cmap=cmap, vmin=0, vmax=n_max)
if ADAPTIVE:
    cbar = fig.colorbar(im, ax=ax, label=r'$N({\bf x})$', ticks=np.arange(0, Nmax + 1, 2))

#
initial_safe_set = lyapunov.initial_safe_set.reshape(disc.num_points)
cmap = ListedColormap([(1., 1., 1., 0.), (1., 0., 0., 1)])
im = ax.imshow(initial_safe_set.T, origin='lower', extent=plot_limits.ravel(), aspect=plot_limits[0, 1] / plot_limits[1, 1], cmap=cmap, vmin=None, vmax=None)

#
if isinstance(lyapunov.dynamics, safe_learning.UncertainFunction):
    # Skip origin data point
    X = norms.ravel() * lyapunov.dynamics.functions[0].X[1:, :disc.ndim]
    ax.plot(X[:, 0], X[:, 1], 'x', color='pink', mew=2, ms=8)
    
#
ax.set_title(r'$M = {}$'.format(disc.num_points[0] - 1)
             + ',  ' + r'$|\mathcal{X}_\tau| =$ ' + r'{:.1e}'.format(disc.nindex)
             + ',  ' + r'$\tau =$ ' + r'{:.0e}'.format(np.sum(disc.unit_maxes) / 2), 
             fontsize=fontsize)
ax.set_xlabel(r'$\phi$ [deg]')
ax.set_ylabel(r'$\dot{\phi}$ [deg/s]')

    
# Legend
# red = initial safe set, pink = data points, grey = true decrease region
colors = ['red', 'pink', 'lightgrey']
proxy = [plt.Rectangle((0,0), 1, 1, fc=c) for c in colors]
labels = [r'Initial safe set $\mathcal{S}^{\!\ 0}_\pi$', r'Measurements ($n = {}$)'.format(len(X)), r'$v(f_{\pi}({\bf x})) - v({\bf x}) < 0$']
ax.legend(proxy, labels, loc='upper right', fontsize=13.5)
    
plt.show()

if OPTIONS.save_figs:
    if ADAPTIVE:
        fig.savefig('figures/pendulum_adaptive_learning_n{}.pdf'.format(len(X)), bbox_inches='tight')
    else:
        fig.savefig('figures/pendulum_learning_n{}.pdf'.format(len(X)), bbox_inches='tight')

In [ ]:
tau1 = np.sum(disc.unit_maxes) / 2
tau2 = 2 / (3000 + 1)

print(tau1)
print(tau2)
print(tau1 / tau2)

In [ ]:
plt.rc('font', size=8)

fig, ax = plt.subplots(1, 2, sharex=False, figsize=(10, 3), dpi=300)
fig.subplots_adjust(wspace=0.3, hspace=0.2)

ax[0].step(data_size, level, 'o', where='post', linestyle='-')
ax[0].set_xlabel(r'Number of data points collected', fontsize=12)
ax[0].set_ylabel(r'$c_{max}$', fontsize=12)

ax[1].step(data_size, safe_size, 'o', where='post')
ax[1].set_xlabel(r'Number of data points collected', fontsize=12)
ax[1].set_ylabel(r'Safe set size', fontsize=12)

plt.show()

print(data_size)
print(level)
print(safe_size)
print(lyapunov.discretization.nindex)
print((dv.eval({states: lyapunov.discretization.all_points}) < 0).sum())

In [ ]:
# debug(lyapunov, true_dynamics, state_norm, do_print=True, newly_safe_only=True)

In [ ]:
#
n = np.array([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120])

# Non-adaptive
cmax = np.array([0.844, 0.844, 0.844, 0.844, 0.844, 0.844, 0.8441, 2.4477, 5.3532, 10.0886, 12.8918, 17.5511, 22.1951])
safe = np.array([206471, 206471, 206471, 206471, 206471, 206471, 206491, 598687, 1309424, 2422172, 2893935, 3527855, 4030337])
grid = 9006001
dv_ = 5585468

# Adaptive
cmax_adapt = np.array([0.844, 2.3248, 5.3878, 10.4285, 15.063, 22.0495, 26.7793, 28.3889, 28.9022, 29.3197, 29.626, 29.9521, 30.1488])
safe_adapt = np.array([5731, 15778, 36609, 69101, 89157, 111739, 123486, 127071, 128153, 129054, 129695, 130374, 130781])
grid_adapt = 251001
dv_adapt = 155482

# Legend
colors = ['red', 'blue']
proxy = [plt.Rectangle((0,0), 1, 1, fc=c) for c in colors]
labels = ['Adaptive', 'Non-adaptive']

#
plt.rc('font', size=15)

fig, ax = plt.subplots(1, 1, figsize=(6, 5), dpi=100)
ax.step(n, cmax, 'bo', where='post', linestyle='--')
ax.step(n, cmax_adapt, 'ro', where='post', linestyle='--')
ax.set_xlabel(r'$n$')
ax.set_ylabel(r'$c_n$')
ax.set_ylim([0, None])
ax.legend(proxy, labels, loc='upper left', fontsize=13.5)
fig.savefig('figures/pendulum_cn.pdf', bbox_inches='tight')

fig, ax = plt.subplots(1, 1, figsize=(6, 5), dpi=100)
ax.step(n, safe / dv_, 'bo', where='post', linestyle='--')
ax.step(n, safe_adapt / dv_adapt, 'ro', where='post', linestyle='--')
ax.set_xlabel(r'$n$')
ax.set_ylabel(r'$|\mathcal{V}(c_n) \cap \mathcal{X}_\tau|\ / \ |\mathcal{D} \cap \mathcal{X}_\tau|$')
ax.set_ylim([0, 1])
ax.legend(proxy, labels, loc='upper left', fontsize=13.5)
fig.savefig('figures/pendulum_safesize.pdf', bbox_inches='tight')

plt.show()